In [ ]:
import pandas as pd

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
# fpath = '/Users/xyuan/Documents/SeqDataSets/Cy3/12_mixtures_5_sulfo_Cy3/20nt-Table 1.csv'
fpath = "/Users/xyuan/Documents/SeqDataSets/Ning1024/12mixtures_3'-Biotin_MFE/19-Table 1.csv"
# fpath = "/Users/xyuan/Documents/SeqDataSets/Ning1024/12mixtures_3'-Biotin_MFE/20-6-Table 1.csv"
df_19 = load_data(fpath, csv_format=True)
df_19 = df_19[df_19.RT<15].copy()
df_19.shape

In [ ]:
plot_zone(df_19)

In [ ]:
df_sample = df_19.copy()
# remove adducts from df_19

adducts = [K, Na]
dfm_adducts_list = list()
for adduct in adducts:
    dfm_r = match_dfs(df_sample, df_sample, shift=adduct)
    dfm_adducts_list.append(dfm_r)
dfm_adducts = pd.concat(dfm_adducts_list).drop_duplicates()
df_without_adducts = df_sample.drop(dfm_adducts.index)
df_without_adducts.shape

In [ ]:
# df_sample = df_without_adducts
df_sample = df_19.copy()

import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

def func(df_sample, shift):
    dfm = match_dfs(df_sample, df_sample, shift=shift)
    sh = ShiftHit(shift=shift, hit=dfm.shape[0])
    return sh

PROCESSES = 16
params = [(df_sample, shift) for shift in np.arange(1, 900, 0.1)]  # biotin 3'
# params = [(df_sample, shift) for shift in np.arange(1, 900, 0.1)] # Sulfo-cy3 5´
# params = [(df_sample, shift) for shift in np.arange(1, 64, 1)]
with multiprocessing.Pool(PROCESSES) as pool:
    shift_hits = pool.starmap(func, params)

df_shift_hits = pd.DataFrame(shift_hits)    

In [ ]:
# fig = px.scatter(df_shift_hits, x='shift', y='hit', labels={'shift': 'Mass Shift', 'hit': 'Hit Count'})
# fig.show()
plt.scatter(df_shift_hits['shift'], df_shift_hits['hit'])

In [ ]:
# df_sh_sulfo = df_shift_hits.copy()
# df_sh_sulfo.to_excel('ShiftHit_Sulfo.xlsx')

In [ ]:
# df_sh_biotin = df_shift_hits.copy()
# df_sh_biotin.to_excel('ShiftHit_Biotin.xlsx')

In [ ]:
df_sh_top = df_shift_hits.sort_values('hit', ascending=False).iloc[:50].copy()

In [ ]:
def func(shift):
    dfm_l, dfm_r = peer_dfs(df_sample, df_sample, shift=shift)
    df = dfm_l if dfm_l.shape[0] < dfm_r.shape[0] else dfm_r
    bcr = base_calling_random(df)
    return len(bcr[1])

df_sh_top['basecallings'] = df_sh_top.apply(lambda x: func(x['shift']), axis=1)

In [ ]:
# df_sh_top.sort_values('basecallings', ascending=False)

In [ ]:
shift = 756.2 #634.1 #A #409 #612.1 #612.1 #biotin 3´
# shift = 834.1828 #sulfo-cy3 5´
df_sample = df_19
dfm_l, dfm_r = peer_dfs(df_sample, df_sample, shift=shift)
dfm_l.shape, dfm_r.shape

In [ ]:
df_sample = df_19.drop(dfm_r.index)

In [ ]:
shift = 22 #660# 327.0 #18 #658.1 #734.2 #756.1954 #biotin 3´
# shift = 834.1828 #sulfo-cy3 5´
df_sample = df_19
dfm_l, dfm_r = peer_dfs(df_sample, df_sample, shift=shift)
dfm_l.shape, dfm_r.shape

In [ ]:
plot_zones(dfm_l, dfm_r)

In [ ]:
bcr = base_calling_random(dfm_l)
plot_basecalling(*bcr)
print(len(bcr[0]))

In [ ]:
bcr_l = base_calling_random(dfm_l)
bcr_r = base_calling_random(dfm_r)
bcr = (pd.concat([bcr_l[0],bcr_r[0]]), bcr_l[1] + bcr_r[1])
# bcr

In [ ]:
# bcr = base_calling_random(dfm_r)
plot_basecalling(*bcr)

In [ ]:
df_top = local_top(df_19, top=1)
df_top.shape

In [ ]:
def draw_df(df, svg_fpath, x='Mass', y='RT', s=6, color='C0', label=''):
#     fig = plt.figure(figsize=(5, 3))
    fig = plt.figure(figsize=(2.5, 1.5))
    plt.scatter(df[x], df[y], s=s, color=color, label=label) 
    plt.legend()

    fig.tight_layout()
    plt.savefig(svg_fpath, transparent=True, dpi=300)
    
def draw_dfs(df1, df2, svg_fpath, x='Mass', y='RT', s=6, colors=['C0', 'r'], labels=['', ''], xy=([], [])):
    fig = plt.figure(figsize=(2.5, 1.5))
    plt.scatter(df1[x], df1[y], s=s, color=colors[0], label=labels[0]) 
    plt.scatter(df2[x], df2[y], s=s, color=colors[1], label=labels[1]) 
    if xy[0]:
        plt.scatter(xy[0], xy[1], color='w')
    plt.legend()

    fig.tight_layout()
    plt.savefig(svg_fpath, transparent=True, dpi=300)
    
def draw_dfs_bar(df1, df2, svg_fpath, x='Mass', y='RT', s=6, colors=['C0', 'r'], labels=['', ''], xy=([], [])):
    fig = plt.figure(figsize=(2.5, 1.5))
    plt.scatter(df1[x], df1[y], s=s, color=colors[0], label=labels[0]) 
    plt.scatter(df2[x], df2[y], s=s, color=colors[1], label=labels[1]) 
    if xy[0]:
        plt.scatter(xy[0], xy[1], color='w')
    plt.legend()

    fig.tight_layout()
    plt.savefig(svg_fpath, transparent=True, dpi=300)
    

In [ ]:
svg_fpath = '/Users/xyuan/Downloads/demo.svg'
draw_df(df_sh_top, svg_fpath, x='shift', y='hit')

In [ ]:
svg_fpath = '/Users/xyuan/Downloads/demo.svg'
draw_df(df_sh_top, svg_fpath, x='shift', y='basecallings')

In [ ]:
# Sample code, plot bar
plotdata = pd.DataFrame({
    "2018":[57,67,77,83],
    "2019":[68,73,80,79],
    "2020":[73,78,80,85]})
plotdata.plot(kind="bar",figsize=(15, 8))
plt.title("FIFA ratings")
plt.xlabel("Footballer")
plt.ylabel("Ratings")

In [ ]:
# df_sh_top = df_sh_top.sort_values('shift')
# df_sh_top[df_sh_top.columns[1:]].plot(kind='bar',figsize=(15, 8))

In [ ]:
fig = plt.figure(figsize=(2.5, 1.5))
# plt.bar(df_sh_top['shift'], df_sh_top[y], s=s, color=colors[0], label=labels[0]) 
# plt.scatter(df2[x], df2[y], s=s, color=colors[1], label=labels[1]) 
# if xy[0]:
#     plt.scatter(xy[0], xy[1], color='w')
df_plt = df_sh_top.sort_values('hit', ascending=False).iloc[:10].copy()
df_plt = df_plt.sort_values('shift')
df_plt1 = df_plt.copy()
df_plt1['shift'] = df_plt1['shift'].astype(int)
df_plt1 = df_plt1.set_index('shift')
df_plt1.plot(kind='bar',figsize=(2.5, 1.5))
plt.legend()

fig.tight_layout()
plt.savefig(svg_fpath, transparent=True, dpi=300)

In [ ]:
fig = plt.figure(figsize=(2.5, 1.5))
plot_basecalling(*bcr, figsize=(5, 3))
plt.legend()

fig.tight_layout()
plt.savefig(svg_fpath, transparent=True, dpi=300)

In [ ]:
fig = plt.figure(figsize=(2.5, 1.5))

plot_zone(df_sample, figsize=(5, 3))

plt.legend()
fig.tight_layout()
plt.savefig(svg_fpath, transparent=True, dpi=300)